In [2]:
import pymongo
import pandas as pd

In [3]:
conn = pymongo.MongoClient()['steam']

In [6]:
cursor = conn['users'].find({})

In [7]:
df = pd.DataFrame(list(cursor))

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 7 columns):
_id                190 non-null object
ban_status         190 non-null object
friends            190 non-null object
game_info          190 non-null object
profile_summary    190 non-null object
steamid            190 non-null object
user_info          190 non-null object
dtypes: object(7)
memory usage: 10.5+ KB


In [9]:
df.head()

,_id,ban_status,friends,game_info,profile_summary,steamid,user_info
0,570e85eb63210f01baa27d46,"{u'NumberOfGameBans': 0, u'NumberOfVACBans': 0...","[{u'steamid': u'76561197986364643', u'friend_s...","[{u'playtime_forever': 2893, u'playtime_2weeks...",Hey,76561197971340950,"{u'timecreated': 1101906077, u'realname': u'Ja..."
1,570e85ee63210f01baa27d47,"{u'NumberOfGameBans': 0, u'NumberOfVACBans': 0...","[{u'steamid': u'76561197960270577', u'friend_s...","[{u'playtime_forever': 0, u'playtime_2weeks': ...",I'm a dude who plays a lot of Team Fortress 2....,76561197988952025,"{u'timecreated': 1174855704, u'realname': u'Mi..."
2,570e85f263210f01baa27d48,"{u'NumberOfGameBans': 0, u'NumberOfVACBans': 0...","[{u'steamid': u'76561197987393100', u'friend_s...","[{u'playtime_forever': 62, u'playtime_2weeks':...",No information given.,76561198169761153,"{u'timecreated': 1419418120, u'realname': u'Be..."
3,570e85f463210f01baa27d49,"{u'NumberOfGameBans': 0, u'NumberOfVACBans': 0...","[{u'steamid': u'76561197960689289', u'friend_s...","[{u'playtime_forever': 19185, u'playtime_2week...",No information given. For those of you asking ...,76561198002922051,"{u'timecreated': 1227069731, u'personaname': u..."
4,570e860063210f01baa27d4a,"{u'NumberOfGameBans': 0, u'NumberOfVACBans': 0...","[{u'steamid': u'76561197966036167', u'friend_s...","[{u'playtime_forever': 1921, u'playtime_2weeks...",murder is just late term abortion,76561198109725137,"{u'timecreated': 1380907328, u'realname': u'da..."


In [10]:
games = df[["steamid", "game_info"]]

In [30]:
df['user_info'][:10]["personaname"]

KeyError: 'personaname'

In [26]:
for game in games["game_info"][0]:
    print game["playtime_forever"], game["name"]

2893 Half-Life 2
1711 Counter-Strike: Source
2090 Half-Life 2: Deathmatch
52 Half-Life 2: Lost Coast
26780 Garry's Mod
33 Deus Ex: Game of the Year Edition
0 Half-Life Deathmatch: Source
217 Half-Life 2: Episode One
1752 Portal
348 Half-Life 2: Episode Two
2857 Grand Theft Auto: Vice City
0 Grand Theft Auto: Vice City
211 Max Payne
172 Max Payne 2: The Fall of Max Payne
1 Grand Theft Auto III
959 Grand Theft Auto: San Andreas
3 Grand Theft Auto
0 Grand Theft Auto 2
0 Grand Theft Auto III
0 Grand Theft Auto: San Andreas
632 Audiosurf
476 BioShock
439 Elf Bowling: Hawaiian Vacation
166 Crysis
86 Grand Theft Auto IV
373 Mirror's Edge
2060 Zeno Clash
85 Judge Dredd: Dredd vs Death
77 Braid
241 Plants vs. Zombies: Game of the Year
64 Machinarium
909 Left 4 Dead 2
0 Left 4 Dead 2 Beta
22 The Misadventures of P.B. Winterbottom
81 Beat Hazard
224 Puzzle Agent
12735 Mafia II
875 Amnesia: The Dark Descent
1440 Worms Reloaded
295 The Polynomial
54 Bloody Good Time
33 World of Goo
13 Iron Grip: Wa

In [32]:
def unpack(df, column, fillna=None):
    ret = None
    if fillna is None:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems()))], axis=1)
        del ret[column]
    else:
        ret = pd.concat([df, pd.DataFrame((d for idx, d in df[column].iteritems())).fillna(fillna)], axis=1)
        del ret[column]
    return ret

#unpack(tf, 'nested', 0)

In [34]:
unpack_user_info = unpack(df, "user_info", fillna=None)

In [36]:
del unpack_user_info["_id"]

In [38]:
unpack_bans = unpack(unpack_user_info, "ban_status", fillna=None)

In [42]:
unpack_bans.head()

,friends,game_info,profile_summary,steamid,avatarfull,commentpermission,communityvisibilitystate,gameextrainfo,gameid,gameserverip,...,primaryclanid,profilestate,realname,timecreated,CommunityBanned,DaysSinceLastBan,EconomyBan,NumberOfGameBans,NumberOfVACBans,VACBanned
0,"[{u'steamid': u'76561197986364643', u'friend_s...","[{u'playtime_forever': 2893, u'playtime_2weeks...",Hey,76561197971340950,https://steamcdn-a.akamaihd.net/steamcommunity...,NaN,3,NaN,NaN,NaN,...,103582791430539083,1.0,Jack,1101906077,False,0,none,0,0,False
1,"[{u'steamid': u'76561197960270577', u'friend_s...","[{u'playtime_forever': 0, u'playtime_2weeks': ...",I'm a dude who plays a lot of Team Fortress 2....,76561197988952025,https://steamcdn-a.akamaihd.net/steamcommunity...,NaN,3,NaN,NaN,NaN,...,103582791432409678,1.0,Mike,1174855704,False,0,none,0,0,False
2,"[{u'steamid': u'76561197987393100', u'friend_s...","[{u'playtime_forever': 62, u'playtime_2weeks':...",No information given.,76561198169761153,https://steamcdn-a.akamaihd.net/steamcommunity...,NaN,3,NaN,NaN,NaN,...,103582791440358174,1.0,Berkay,1419418120,False,0,none,0,0,False
3,"[{u'steamid': u'76561197960689289', u'friend_s...","[{u'playtime_forever': 19185, u'playtime_2week...",No information given. For those of you asking ...,76561198002922051,https://steamcdn-a.akamaihd.net/steamcommunity...,NaN,3,NaN,NaN,NaN,...,103582791429555685,1.0,NaN,1227069731,False,0,none,0,0,False
4,"[{u'steamid': u'76561197966036167', u'friend_s...","[{u'playtime_forever': 1921, u'playtime_2weeks...",murder is just late term abortion,76561198109725137,https://steamcdn-a.akamaihd.net/steamcommunity...,1.0,3,NaN,NaN,NaN,...,103582791439431790,1.0,dank meme master,1380907328,False,0,none,0,0,False


In [50]:
for game in games["game_info"][10]:
    print game["name"], game["playtime_forever"]

Counter-Strike: Source 1921
Garry's Mod 28282
Killing Floor 1089
Killing Floor Mod: Defence Alliance 2 0
Call of Duty: Modern Warfare 2 4
Call of Duty: Modern Warfare 2 - Multiplayer 335
Left 4 Dead 2 1083
Left 4 Dead 2 Beta 0
Arma 2 160
Arma 2: Operation Arrowhead 1135
Arma 2: Operation Arrowhead Beta (Obsolete) 0
Terraria 22636
Sanctum 193
Call of Duty: Modern Warfare 3 72
Call of Duty: Modern Warfare 3 - Multiplayer 755
The Binding of Isaac 224
The Elder Scrolls V: Skyrim 1317
The Ship 247
The Ship Single Player 216
The Ship Tutorial 12
Borderlands 2 870
A.V.A - Alliance of Valiant Arms 0
Realm of the Mad God 3596
Loadout 145
Dungeonland 0
Warframe 23
War Thunder 1
Tom Clancy's Ghost Recon Phantoms - NA 26332
Moonbase Alpha 245
Toribash 783
Fistful of Frags 365
You Have to Win the Game 43
No More Room in Hell 230
theHunter 9
Arma 2: DayZ Mod 2
Surgeon Simulator 1102
Sanctum 2 5
The Forest 801
SpeedRunners 248
Risk of Rain 219
PAYDAY 2 5290
Outlast 261
7 Days to Die 1525
Rust 19682
A

In [166]:
few_games = games[:100]

In [94]:
IdSeries = few_games["steamid"]

In [95]:
few_games

,steamid,game_info
0,76561197971340950,"[{u'playtime_forever': 2893, u'playtime_2weeks..."
1,76561197988952025,"[{u'playtime_forever': 0, u'playtime_2weeks': ..."
2,76561198169761153,"[{u'playtime_forever': 62, u'playtime_2weeks':..."
3,76561198002922051,"[{u'playtime_forever': 19185, u'playtime_2week..."
4,76561198109725137,"[{u'playtime_forever': 1921, u'playtime_2weeks..."
5,76561198085049223,"[{u'playtime_forever': 0, u'playtime_2weeks': ..."
6,76561197971340950,"[{u'playtime_forever': 2893, u'playtime_2weeks..."
7,76561197988952025,"[{u'playtime_forever': 0, u'playtime_2weeks': ..."
8,76561198169761153,"[{u'playtime_forever': 62, u'playtime_2weeks':..."
9,76561198002922051,"[{u'playtime_forever': 19185, u'playtime_2week..."


In [168]:
for i, game in enumerate(few_games["game_info"]):
    for g in game:
        g["uid"] = few_games["steamid"][i]
        #del g["appid"]
    #print game[0]

In [121]:
few_games["game_info"][0]

[{u'name': u'Half-Life 2',
  u'playtime_2weeks': 0,
  u'playtime_forever': 2893,
  'uid': u'76561197971340950'},
 {u'name': u'Counter-Strike: Source',
  u'playtime_2weeks': 0,
  u'playtime_forever': 1711,
  'uid': u'76561197971340950'},
 {u'name': u'Half-Life 2: Deathmatch',
  u'playtime_2weeks': 0,
  u'playtime_forever': 2090,
  'uid': u'76561197971340950'},
 {u'name': u'Half-Life 2: Lost Coast',
  u'playtime_2weeks': 0,
  u'playtime_forever': 52,
  'uid': u'76561197971340950'},
 {u'name': u"Garry's Mod",
  u'playtime_2weeks': 43,
  u'playtime_forever': 26780,
  'uid': u'76561197971340950'},
 {u'name': u'Deus Ex: Game of the Year Edition',
  u'playtime_2weeks': 0,
  u'playtime_forever': 33,
  'uid': u'76561197971340950'},
 {u'name': u'Half-Life Deathmatch: Source',
  u'playtime_2weeks': 0,
  u'playtime_forever': 0,
  'uid': u'76561197971340950'},
 {u'name': u'Half-Life 2: Episode One',
  u'playtime_2weeks': 0,
  u'playtime_forever': 217,
  'uid': u'76561197971340950'},
 {u'name': u'Po

In [135]:
GameSeries = few_games["game_info"][1]
GameSeries2 = pd.DataFrame(few_games["game_info"][2])

In [115]:
SparseGames = pd.DataFrame(GameSeries)

In [177]:
SparseGames.head()

,appid,name,playtime_2weeks,playtime_forever,uid
0,NaN,Half-Life 2,0.0,2893.0,76561197971340950
1,NaN,Counter-Strike: Source,0.0,1711.0,76561197971340950
2,NaN,Half-Life 2: Deathmatch,0.0,2090.0,76561197971340950
3,NaN,Half-Life 2: Lost Coast,0.0,52.0,76561197971340950
4,NaN,Garry's Mod,43.0,26780.0,76561197971340950


In [125]:
for u in few_games["game_info"]:
    user_df = pd.DataFrame(u)
    SparseGames.append(user_df)

In [170]:
for u in few_games["game_info"]:
    Gamedf = pd.DataFrame(u)
    SparseGames = SparseGames.append(Gamedf)

In [169]:
SparseGames = pd.DataFrame(columns=["name", "playtime_2weeks", "playtime_forever", "uid"])

In [171]:
SparseGames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10497 entries, 0 to 110
Data columns (total 5 columns):
appid               9305 non-null float64
name                10497 non-null object
playtime_2weeks     10497 non-null float64
playtime_forever    10497 non-null float64
uid                 10497 non-null object
dtypes: float64(3), object(2)
memory usage: 492.0+ KB


In [172]:
# df.pivot(index='date', columns='variable', values='value')
Pivoted = pd.pivot_table(SparseGames, index="uid", columns=["name"], values=["playtime_forever"], fill_value=0)

In [175]:
Pivoted.head()

playtime_forever  \
name                     0RBITALIS   
uid                                  
76561197960548341                0   
76561197960869933                0   
76561197961992433                0   
76561197962141393                0   
76561197962190036                0   

                                                                              \
name              1... 2... 3... KICK IT! (Drop That Beat Like an Ugly Baby)   
uid                                                                            
76561197960548341                                                  0           
76561197960869933                                                  0           
76561197961992433                                                  0           
76561197962141393                                                  0           
76561197962190036                                                  0           

                                                                          \
name              10 Second Ninja 10,000,000 100% Orange Juice 1000 Amps   
uid                                                                        
76561197960548341               0          0                 0         0   
76561197960869933               0          0                 0         0   
76561197961992433               0          0                 0         0   
76561197962141393               0          0                 0         0   
76561197962190036               0          0                 0         0   

                                                                      ...     \
name              1Quest 20XX 3089 -- Futuristic Action RPG 3DMark    ...      
uid                                                                   ...      
76561197960548341      0    0                             0     71    ...      
76561197960869933      0    0                             0      0    ...      
76561197961992433      0    0                             0      0    ...      
76561197962141393      0    0                             0      0    ...      
76561197962190036      0    0                             0      0    ...      

                                                                         \
name              Zombies Monsters Robots Zuma's Revenge bit Dungeon II   
uid                                                                       
76561197960548341                       0              0              0   
76561197960869933                       0              0              0   
76561197961992433                       0              0              0   
76561197962141393                       0              0              0   
76561197962190036                       0              0              0   

                                                                  \
name              eXceed - Gun Bullet Children how do you Do It?   
uid                                                                
76561197960548341                            0                 0   
76561197960869933                            0                 0   
76561197961992433                            0                 0   
76561197962141393                            0                 0   
76561197962190036                            0                 0   

                                                                         \
name              planetarian ~the reverie of a little planet~ realMyst   
uid                                                                       
76561197960548341                                            0        0   
76561197960869933                                            0        0   
76561197961992433                                            0        0   
76561197962141393                                            0        0   
76561197962190036                                            0        0   

                                                                        
na